# 🏥 SimuPaciente UMH - Demo en Google Colab

**Universidad Miguel Hernández de Elche**

Sistema de simulación de pacientes virtuales con voz para práctica de entrevistas clínicas.

---

## 📋 Instrucciones

1. **Ejecuta las celdas en orden** (Shift + Enter)
2. **Proporciona tu API key de OpenAI** cuando se te solicite
3. **Accede a la URL de ngrok** que se generará al final
4. **¡Comienza a practicar!**

⚠️ **IMPORTANTE:** Este notebook usa recursos de Colab gratuitos. La sesión se cerrará automáticamente después de ~12 horas de inactividad.

---

## 🔧 Paso 1: Instalación de Dependencias

Instalamos Python, Node.js y todas las dependencias necesarias.

In [ ]:
%%bash
echo "📦 Instalando dependencias del sistema..."

# Actualizar sistema
apt-get update -qq

# Instalar Node.js 18.x
curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
apt-get install -y nodejs

# Verificar versiones
echo "✅ Node.js version: $(node --version)"
echo "✅ npm version: $(npm --version)"
echo "✅ Python version: $(python3 --version)"

## 📥 Paso 2: Clonar Repositorio

Descargamos el código desde GitHub.

In [ ]:
import subprocess
import os

print("📥 Clonando repositorio desde GitHub...")

# Ir a /content primero
os.chdir('/content')

# Limpiar si existe
if os.path.exists('/content/ECOE'):
    subprocess.run(['rm', '-rf', 'ECOE'], check=True)

# Clonar repo
result = subprocess.run(
    ['git', 'clone', 'https://github.com/marcosbenghezala/ECOE.git', 'ECOE'],
    capture_output=True,
    text=True
)

if result.returncode != 0:
    print("❌ Error clonando repositorio:")
    print(result.stderr)
    raise RuntimeError("Failed to clone repository")

print("✅ Repositorio clonado exitosamente")

# Verificar que existe
if os.path.exists('/content/ECOE/simulador'):
    print("✅ Directorio /content/ECOE/simulador verificado")
    # Listar contenido
    result = subprocess.run(['ls', '-la', '/content/ECOE/'], capture_output=True, text=True)
    print(result.stdout)
else:
    print("❌ ERROR: El directorio no existe después de clonar")
    raise RuntimeError("Clone verification failed")

## 🐍 Paso 3: Instalar Dependencias Python

Instalamos las librerías de Python necesarias para el backend.

In [ ]:
%%bash
set -e  # Fallar si cualquier comando falla

# Verificar que el directorio existe
if [ ! -d "/content/ECOE/simulador" ]; then
    echo "❌ ERROR: El directorio /content/ECOE/simulador no existe"
    echo "⚠️  Asegúrate de haber ejecutado la celda de clonado (Paso 2) primero"
    exit 1
fi

cd /content/ECOE/simulador

echo "🐍 Instalando dependencias Python..."
pip install -q -r requirements.txt

echo "✅ Dependencias Python instaladas"

## ⚛️ Paso 4: Build del Frontend

Compilamos el frontend React.

In [ ]:
%%bash
set -e  # Fallar si cualquier comando falla

# Verificar que el directorio existe
if [ ! -d "/content/ECOE/simulador/frontend" ]; then
    echo "❌ ERROR: El directorio /content/ECOE/simulador/frontend no existe"
    echo "⚠️  Asegúrate de haber ejecutado la celda de clonado (Paso 2) primero"
    exit 1
fi

cd /content/ECOE/simulador/frontend

echo "⚛️ Instalando dependencias de Node.js..."
npm install --silent

echo "🔨 Building frontend..."
npm run build

echo "✅ Frontend compilado exitosamente"
ls -la /content/ECOE/simulador/frontend/dist/

## 🔑 Paso 5: Configuración Automática

**AUTOMÁTICO** - Los estudiantes NO necesitan hacer nada aquí.

La API key de OpenAI está configurada en el servidor proxy (oculta y segura).

In [ ]:
import os

print("="*70)
print("✅ CONFIGURACIÓN AUTOMÁTICA")
print("="*70)

# URL del proxy server (desplegado en Render.com con API key oculta)
PROXY_URL = "https://simu-paciente-umh-proxy.onrender.com"

# Configurar para usar el proxy
os.environ['PROXY_URL'] = PROXY_URL

print(f"\n🔒 Proxy server: {PROXY_URL}")
print("")
print("📝 Información:")
print("   - La API key de OpenAI está en el servidor proxy (oculta)")
print("   - Los estudiantes NO pueden ver ni copiar la API key")
print("   - El uso está controlado a través del proxy")
print("")
print("👉 Continúa ejecutando las siguientes celdas")
print("="*70)

## 🌐 Paso 6: Instalar y Configurar ngrok

ngrok nos permite exponer el servidor local a Internet de forma segura.

In [ ]:
%%bash
echo "🌐 Instalando Cloudflare Tunnel (cloudflared)..."

# Descargar cloudflared (alternativa a ngrok, sin autenticación)
wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
chmod +x cloudflared
mv cloudflared /usr/local/bin/

echo "✅ Cloudflare Tunnel instalado"
cloudflared --version

echo ""
echo "ℹ️  Usando Cloudflare Tunnel en lugar de ngrok"
echo "   - No requiere autenticación"
echo "   - Completamente gratuito"
echo "   - Sin límites de túneles"

## 🚀 Paso 7a: Iniciar Servidor Flask

Iniciamos el servidor en background.

In [ ]:
import subprocess
import time
import os

print("🚀 Iniciando servidor...")
print("="*70)

os.chdir('/content/ECOE/simulador')

# Limpiar procesos anteriores
print("\n🧹 Limpiando procesos...")
subprocess.run(['pkill', '-9', '-f', 'colab_server.py'], stderr=subprocess.DEVNULL)
subprocess.run(['pkill', '-9', '-f', 'cloudflared'], stderr=subprocess.DEVNULL)
subprocess.run(['fuser', '-k', '8080/tcp'], stderr=subprocess.DEVNULL, stdout=subprocess.DEVNULL)
time.sleep(2)
print("✅ Procesos limpiados")

# Iniciar servidor
print("\n🏥 Iniciando servidor Flask...")
with open('/tmp/flask.log', 'w') as f:
    subprocess.Popen(['python3', 'colab_server.py'], stdout=f, stderr=subprocess.STDOUT)

print("✅ Servidor iniciado en background")
print("📊 Logs en: /tmp/flask.log")

In [ ]:
import time
import re
from IPython.display import display, HTML

print("🔍 Obteniendo URL pública...")

public_url = None
for i in range(8):
    try:
        with open('/tmp/cloudflared.log', 'r') as f:
            content = f.read()
            match = re.search(r'(https://[a-z0-9-]+\.trycloudflare\.com)', content)
            if match:
                public_url = match.group(1)
                print(f"✅ URL encontrada!")
                break
    except:
        pass
    
    if i < 7:
        print(f"   Intento {i+1}/8...")
        time.sleep(3)

if public_url:
    print("\n" + "="*70)
    print("✅ ✅ ✅  LISTO  ✅ ✅ ✅")
    print("="*70)
    print(f"\n🌍 URL: {public_url}\n")
    
    display(HTML(f'<a href="{public_url}" target="_blank" style="font-size:20px;padding:15px 30px;background:#667eea;color:white;text-decoration:none;border-radius:10px;display:inline-block;">🚀 Abrir App</a>'))
else:
    print("\n❌ No se obtuvo URL")
    print("💡 Ejecuta esta celda de nuevo")

In [ ]:
import subprocess
import time
import requests

print("⏳ Esperando servidor (15s)...")
time.sleep(15)

# Verificar servidor
print("\n🔍 Verificando servidor...")
for i in range(5):
    try:
        requests.get('http://localhost:8080/api/cases', timeout=3)
        print("✅ Servidor OK")
        break
    except:
        if i < 4:
            print(f"   Reintento {i+1}/5...")
            time.sleep(3)
        else:
            print("⚠️ Servidor no responde, pero continuamos...")

# Iniciar cloudflared
print("\n🌐 Iniciando Cloudflare Tunnel...")
with open('/tmp/cloudflared.log', 'w') as f:
    subprocess.Popen(['cloudflared', 'tunnel', '--url', 'http://localhost:8080'], stdout=f, stderr=subprocess.STDOUT)

print("✅ Túnel iniciado")
print("⏳ Espera 10s y ejecuta la siguiente celda...")

## 🚀 Paso 7c: Obtener URL Pública

¡Última celda! Obtiene la URL para acceder a la aplicación.

## 🚀 Paso 7b: Crear Túnel Público

Creamos el túnel con Cloudflare (espera ~20 segundos).

---

## 📝 Notas Adicionales

### 🔄 Reiniciar el Servidor
Si necesitas reiniciar:
1. Detén la celda anterior (botón Stop)
2. Ejecuta la celda nuevamente

### 🐛 Solución de Problemas

**Problema:** No se conecta a OpenAI Realtime API
- Verifica que tu API key tiene acceso a Realtime API (beta)
- Comprueba que tienes créditos disponibles

**Problema:** La URL de ngrok no funciona
- Espera 30 segundos y recarga la página
- Verifica que la celda del servidor sigue corriendo [*]

### 📧 Soporte
- GitHub: https://github.com/marcosbenghezala/ECOE
- Universidad Miguel Hernández de Elche

---

**Versión:** 0.9.5 (Release Candidate)  
**Última actualización:** 17 de diciembre de 2025